In [2]:
def CityChoice(workspace, USA_NPP, NPP_D, USA_NTS, NTS_D, USA_GPP, GPP_D, USA_CPP, CPP_D, Population, Vacancy, Field, Method):
    import arcpy
    arcpy.env.overwriteOutput = True
    arcpy.env.workspace = "C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/City_Choice_In_Python.gdb"
    workspace = arcpy.env.workspace
    
    intermediate_layers = ["USA_NPP_Buffer","USA_NTS_Buffer","USA_Nuclear_Risk","USA_GPP_Buffer","USA_CPP_Buffer",
                           "USA_Human_Activities_Air_Pollution","USA_Total_Risks_Areas","Less_Polluted_Cities", 
                           "Les_Polluted_Cities_Population", "Less_Polluted_Cities_Population_Vacancy",
                           "Less_Polluted_Cities_Population_Vacancy_Ranked"]
    
    for intermediate_layer in intermediate_layers:
        if arcpy.Exists(intermediate_layer):
            arcpy.Delete_management(intermediate_layer)



    # Buffers around Nuclear Power Plants
    arcpy.Buffer_analysis("C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/data/USA_NPP.shp", 
                          "USA_NPP_Buffer", NPP_D,"","", "ALL","")


    # Buffers around Nuclear Tests Sites
    arcpy.Buffer_analysis("C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/data/USA_NTS.shp", 
                          "USA_NTS_Buffer", NTS_D,"","", "ALL","")
    
    # Union of buffers around Nuclear Tests Sites & Nuclear Power Plants (Nuclear Risks)
    inFeatures = [["USA_NPP_Buffer"], ["USA_NTS_Buffer"]]
    arcpy.Union_analysis(inFeatures, "USA_Nuclear_Risk")

    # Buffers around Gas Power Plants
    arcpy.Buffer_analysis("C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/data/USA_GPP.shp", 
                          "USA_GPP_Buffer", GPP_D,"","", "ALL","")
    
    # Buffers around Coal Power Plants
    arcpy.Buffer_analysis("C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/data/USA_CPP.shp", 
                          "USA_CPP_Buffer", CPP_D,"","", "ALL","")

    # Union of buffers around Gas & Coal Power Plants (Human Risks)
    inFeatures = [["USA_CPP_Buffer"], ["USA_GPP_Buffer"]]
    arcpy.Union_analysis(inFeatures, "USA_Human_Activities_Air_Pollution")

    # Union of all buffers (Total Risk USA)
    inFeatures = [["USA_Nuclear_Risk"], ["USA_Human_Activities_Air_Pollution"]]
    arcpy.Union_analysis(inFeatures, "USA_Total_Risks_Areas")
    
    # Remove all the cities that are into USA_Total_Risks_Areas
    inFeatures = "C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/Major_Cities_US/USA_Major_Cities.shp"
    erase_features = "USA_Total_Risks_Areas"
    arcpy.Erase_analysis(inFeatures, erase_features, "Less_Polluted_Cities")
    
    # Select only cities that are outside the risks areas 
    # AND have a population count higher of a user entered parameter
    inFeatures = "Less_Polluted_Cities"
    outLocation = "C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/City_Choice_In_Python.gdb"
    outFeatureClass = "Less_Polluted_Cities_Population"
    delimitedField = arcpy.AddFieldDelimiters("C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/Major_Cities_US/USA_Major_Cities.shp",
                                              "POPULATION")
    expression = "POPULATION >" + str(Population)
    arcpy.FeatureClassToFeatureClass_conversion(inFeatures, outLocation, outFeatureClass, expression)
    
    # Select only cities that are outside the risks areas 
    # AND have a population count higher of a user entered parameter
    # AND have a vacancy rate higher than a user entered parameter
    inFeatures = "Less_Polluted_Cities_Population"
    outLocation = "C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/City_Choice_In_Python.gdb"
    outFeatureClass = "Less_Polluted_Cities_Population_Vacancy"
    delimitedField = arcpy.AddFieldDelimiters("C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/Major_Cities_US/USA_Major_Cities.shp",
                                              "VACANT")
    expression = "VACANT >" + str(Vacancy)
    arcpy.FeatureClassToFeatureClass_conversion(inFeatures, outLocation, outFeatureClass, expression)
    
    # Sorting all cities from the final table according to their population
    in_dataset = "Less_Polluted_Cities_Population_Vacancy"
    out_dataset = "Less_Polluted_Cities_Population_Vacancy_Ranked"
    sort_field = [[str(Field), str(Method)]]
    sort_method = "PEANO" # space filling curve algorithm
    arcpy.Sort_management(in_dataset, out_dataset, sort_field, sort_method)

In [3]:
CityChoice("C:/Thomas_Calvet_City_Choice/City_Choice_In_Python/City_Choice_In_Python.gdb",
           "USA_NPP", "50 miles",
           "USA_NTS", "150 miles",
           "USA_GPP", "30 miles", 
           "USA_CPP", "30 miles",
           100000, 28000,
          "POPULATION", "DESCENDING")